### Student Information
Name: 許宸瑄

Student ID: B10209001

GitHub ID: ChenHsuanHsu

Kaggle name: kyonatsuki

Kaggle private scoreboard snapshot: 

![Scoreboard](scoreboard.png)

---

### Instructions

1. First: __This part is worth 30% of your grade.__ Do the **take home** exercises in the [DM2024-Lab2-master Repo](https://github.com/didiersalazar/DM2024-Lab2-Master). You may need to copy some cells from the Lab notebook to this notebook. 


2. Second: __This part is worth 30% of your grade.__ Participate in the in-class [Kaggle Competition](https://www.kaggle.com/competitions/dm-2024-isa-5810-lab-2-homework) regarding Emotion Recognition on Twitter by this link: https://www.kaggle.com/competitions/dm-2024-isa-5810-lab-2-homework. The scoring will be given according to your place in the Private Leaderboard ranking: 
    - **Bottom 40%**: Get 20% of the 30% available for this section.

    - **Top 41% - 100%**: Get (60-x)/6 + 20 points, where x is your ranking in the leaderboard (ie. If you rank 3rd your score will be (60-3)/6 + 20 = 29.5% out of 30%)   
    Submit your last submission **BEFORE the deadline (Nov. 26th 11:59 pm, Tuesday)**. Make sure to take a screenshot of your position at the end of the competition and store it as '''pic0.png''' under the **img** folder of this repository and rerun the cell **Student Information**.
    

3. Third: __This part is worth 30% of your grade.__ A report of your work developping the model for the competition (You can use code and comment it). This report should include what your preprocessing steps, the feature engineering steps and an explanation of your model. You can also mention different things you tried and insights you gained. 


4. Fourth: __This part is worth 10% of your grade.__ It's hard for us to follow if your code is messy :'(, so please **tidy up your notebook**.


Upload your files to your repository then submit the link to it on the corresponding e-learn assignment.

Make sure to commit and save your changes to your repository __BEFORE the deadline (Nov. 26th 11:59 pm, Tuesday)__. 

---
### Begin Assignment Here

1. First part of my homework is done in DM2024-Lab2-Master.ipynb under DM2024-Lab2-Master file [open Notebook](https://github.com/ChenHsuanHsu/DM2024-Lab2-Master/blob/main/DM2024-Lab2-Master.ipynb)

2. Second part of my homework is Kaggle.ipynb under same files as this text
[open Notebook](https://github.com/ChenHsuanHsu/DM2024-Lab2-Homework/blob/main/Kaggle.ipynb)

3. Third: A report of your work developping the model for the competition (You can use code and comment it). This report should include what your preprocessing steps, the feature engineering steps and an explanation of your model. You can also mention different things you tried and insights you gained. 
撰寫一份您參加比賽時開發模型的報告（可以使用並註解代碼）。此報告應包含您的數據預處理步驟、特徵工程步驟以及對模型的解釋。您還可以提及您嘗試過的不同方法以及獲得的見解。

## 1. data preprocessing







讀取json所有欄位，存成所有tweets的dataframe。再將其與情緒分類檔案及身份識別文件（訓練或測試資料）合併。  
Read all fields from the JSON file and store them as a DataFrame containing all tweets. Then, merge this DataFrame with the emotion classification file and the identity file (indicating whether the data is for training or testing).

In [ ]:
import json
import pandas as pd

# 存储提取数据的列表
tweets_data = []

# 读取 JSON 文件
with open('dm-2024-isa-5810-lab-2-homework/tweets_DM.json', 'r') as file:
    for line in file:
        try:
            # 解析 JSON 每行
            tweet = json.loads(line)
            
            # 提取字段
            tweet_id = tweet["_source"]["tweet"].get("tweet_id")
            text = tweet["_source"]["tweet"].get("text")
            hashtags = tweet["_source"]["tweet"].get("hashtags", [])
            crawldate = tweet.get("_crawldate")
            score = tweet.get("_score")
            index = tweet.get("_index")
            doc_type = tweet.get("_type")
            
            # 衍生字段
            hashtag_count = len(hashtags)
            text_length = len(text) if text else 0
            contains_keyword = "<LH>" in text if text else False

            # 将提取的字段存入字典，添加到列表
            tweets_data.append({
                "tweet_id": tweet_id,
                "text": text,
                "hashtags": hashtags,
                "crawldate": crawldate,
                "_score": score,
                "_index": index,
                "_type": doc_type,
                "hashtag_count": hashtag_count,
                "text_length": text_length,
                "contains_keyword": contains_keyword
            })
        except json.JSONDecodeError as e:
            print("JSON decode error:", e)

# 转换为 DataFrame
df = pd.DataFrame(tweets_data)

classify = pd.read_csv('dm-2024-isa-5810-lab-2-homework/data_identification.csv')  # 包含 tweet_id 和 identification
emotion = pd.read_csv('dm-2024-isa-5810-lab-2-homework/emotion.csv')  # 包含 tweet_id 和 emotion
data = pd.merge(df, classify, on='tweet_id', how='inner')
data = pd.merge(data, emotion, on='tweet_id', how='outer')
# data # 將情緒特徵及資料分類欄位加入相同dataframe

# 將 data_identification 中 identification 欄位值為 'train' 的選出來作為訓練集
train_data = data[data['identification'] == 'train']

# 將 identification 欄位值為 'test' 的選出來作為測試集
test_data = data[data['identification'] == 'test']

# 所有情感emotion in emotion.csv
emotions = data['emotion'].unique()

## 2. feature enginering

### vectorize the text in each tweets 將文字資料轉成向量形式

#### Bag of Word

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
import nltk

# build analyzers (bag-of-words)
BOW_500 = CountVectorizer(max_features=500, tokenizer=nltk.word_tokenize) 

# apply analyzer to training data
BOW_500.fit(train_data['text'])

X_train = BOW_500.transform(train_data['text'])

y_train = train_data['emotion']

X_test = BOW_500.transform(test_data['text'])
train_data

#### TF-IDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# 初始化 TfidfVectorizer，并限制特征数为 1000
tfidf_vect = TfidfVectorizer(max_features=1000)

# 使用 TF-IDF 向量化器进行训练集和测试集的向量化
X_train = tfidf_vect.fit_transform(train_data["text"])
X_test = tfidf_vect.transform(test_data["text"])

# 获取特征名称列表
feature_names = tfidf_vect.get_feature_names_out()

## 3. model

### MLP

In [ ]:
from sklearn.neural_network import MLPClassifier
import pandas as pd

# 讀取 .pkl 文件
test_data = pd.read_pickle('output_1/test_data.pkl')
X_train = pd.read_pickle('output_3/X_train.pkl')
y_train = pd.read_pickle('output_3/y_train.pkl')
X_test = pd.read_pickle('output_3/X_test.pkl')
y_test = pd.read_pickle('output_3/y_test.pkl')

# 將特徵轉換為浮點型
X_train = X_train.astype(float)
X_test = X_test.astype(float)

# 初始化並訓練多層感知機模型
mlp_clf = MLPClassifier(random_state=42, max_iter=500)
mlp_clf.fit(X_train, y_train)

# 進行預測
y_pred = mlp_clf.predict(X_test)

# 保存結果至 CSV
result_df = pd.DataFrame({
    'id': test_data['tweet_id'],
    'emotion': y_pred
})

result_df.to_csv("prediction_results.csv", index=False)
print("Prediction results saved to prediction_results.csv")

### K nearist

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
import pandas as pd

# 讀取 .pkl 文件
test_data = pd.read_pickle('output_1/test_data.pkl')
X_train = pd.read_pickle('output_3/X_train.pkl')
y_train = pd.read_pickle('output_3/y_train.pkl')
X_test = pd.read_pickle('output_3/X_test.pkl')
y_test = pd.read_pickle('output_3/y_test.pkl')

# 將特徵轉換為浮點型
X_train = X_train.astype(float)
X_test = X_test.astype(float)

# 初始化並訓練 KNN 模型
knn_clf = KNeighborsClassifier()
knn_clf.fit(X_train, y_train)

# 進行預測
y_pred = knn_clf.predict(X_test)

# 保存結果至 CSV
result_df = pd.DataFrame({
    'id': test_data['tweet_id'],
    'emotion': y_pred
})

result_df.to_csv("prediction_results.csv", index=False)
print("Prediction results saved to prediction_results.csv")

KeyboardInterrupt: 

### Logistic

In [ ]:
import pandas as pd
from sklearn.linear_model import LogisticRegression

# 讀取 .pkl 文件
test_data = pd.read_pickle('output_1/test_data.pkl')
X_train = pd.read_pickle('output_3/X_train.pkl')
y_train = pd.read_pickle('output_3/y_train.pkl')
X_test = pd.read_pickle('output_3/X_test.pkl')
y_test = pd.read_pickle('output_3/y_test.pkl')

# 將特徵轉換為浮點型
X_train = X_train.astype(float)
X_test = X_test.astype(float)

# 初始化並訓練邏輯迴歸模型
lr_clf = LogisticRegression(max_iter=1000, random_state=42)
lr_clf.fit(X_train, y_train)

# 進行預測
y_pred = lr_clf.predict(X_test)

# 保存結果至 CSV
result_df = pd.DataFrame({
    'id': test_data['tweet_id'],
    'emotion': y_pred
})

result_df.to_csv("prediction_results.csv", index=False)
print("Prediction results saved to prediction_results.csv")

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Prediction results saved to prediction_results.csv


### CatBoost

In [ ]:
import pandas as pd
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score

# 讀取 .pkl 文件
test_data = pd.read_pickle('output_1/test_data.pkl')
X_train = pd.read_pickle('output_3/X_train.pkl')
y_train = pd.read_pickle('output_3/y_train.pkl')
X_test = pd.read_pickle('output_3/X_test.pkl')
y_test = pd.read_pickle('output_3/y_test.pkl')

# 將特徵轉換為浮點型
X_train = X_train.astype(float)
X_test = X_test.astype(float)

# 初始化並訓練 CatBoost 模型
catboost_clf = CatBoostClassifier(random_state=42, verbose=0)  # verbose=0 關閉訓練輸出
catboost_clf.fit(X_train, y_train)

# 進行預測
y_pred = catboost_clf.predict(X_test)

# 保存結果至 CSV
result_df = pd.DataFrame({
    'id': test_data['tweet_id'],
    'emotion': y_pred
})

result_df.to_csv("prediction_results.csv", index=False)
print("Prediction results saved to prediction_results.csv")

### XGB

In [ ]:
import pandas as pd
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

# 讀取 .pkl 文件
X_train = pd.read_pickle('output_3/X_train.pkl')
y_train = pd.read_pickle('output_3/y_train.pkl')
X_test = pd.read_pickle('output_3/X_test.pkl')
y_test = pd.read_pickle('output_3/y_test.pkl')

# 將特徵轉換為浮點型
X_train = X_train.astype(float)
X_test =X_test.astype(float)

# 初始化並訓練 XGBoost 模型
xgb_clf = XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='mlogloss')
xgb_clf.fit(X_train, y_train)

# 進行預測
y_pred = xgb_clf.predict(X_test)
y_pred

/opt/anaconda3/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [05:31:30] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
# 將數字標籤轉換回原始情緒標籤
y_pred_decoded = label_encoder.inverse_transform(y_pred)

# 輸出解碼後的情緒標籤
print(y_pred_decoded)
len(y_pred_decoded)

# 保存結果至 CSV
result_df = pd.DataFrame({
    'id': test_data["tweet_id"],
    'predicted_emotion': y_pred_decoded
})

result_df.to_csv("prediction_results.csv", index=False)
print("Prediction results saved to prediction_results.csv")

Prediction results saved to prediction_results.csv


### RandomForest

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# 讀取 .pkl 文件
X_train = pd.read_pickle('output_3/X_train.pkl')
y_train = pd.read_pickle('output_3/y_train.pkl')
X_test = pd.read_pickle('output_3/X_test.pkl')
y_test = pd.read_pickle('output_3/y_test.pkl')

# 將特徵轉換為浮點型
X_train = X_train.astype(float)
X_test = X_test.astype(float)

# 初始化並訓練隨機森林模型
rf_clf = RandomForestClassifier(random_state=42)
rf_clf.fit(X_train, y_train)

# 進行預測
y_pred = rf_clf.predict(X_test)

In [ ]:
# 計算預測精度
accuracy = accuracy_score(y_test, y_pred)
print(f"Random Forest Accuracy: {accuracy:.4f}")

# 保存結果至 CSV
result_df = pd.DataFrame({
    'id': X_test['tweet_id'],
    'predicted_emotion': y_pred
})

result_df.to_csv("prediction_results_randomforest.csv", index=False)
print("Prediction results saved to prediction_results_randomforest.csv")

### 優化模型

In [ ]:
from sklearn.model_selection import GridSearchCV

# 定義超參數範圍
param_grid = {
    'num_leaves': [31, 50, 70],
    'learning_rate': [0.01, 0.05, 0.1],
    'n_estimators': [100, 200, 500],
    'max_depth': [10, 20, 30]
}

# 使用 GridSearchCV 進行參數搜索
grid_search = GridSearchCV(
    estimator=LGBMClassifier(random_state=42),
    param_grid=param_grid,
    scoring='accuracy',
    cv=3
)

grid_search.fit(X_train, y_train)

# 使用最佳參數
best_lgbm_clf = grid_search.best_estimator_
print("Best parameters found: ", grid_search.best_params_)

# 在最佳參數下訓練模型並預測
best_lgbm_clf.fit(X_train, y_train)
y_pred = best_lgbm_clf.predict(X_test)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.595803 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2312
[LightGBM] [Info] Number of data points in the train set: 970375, number of used features: 500
[LightGBM] [Info] Start training from score -3.597599
[LightGBM] [Info] Start training from score -1.765960
[LightGBM] [Info] Start training from score -2.347947
[LightGBM] [Info] Start training from score -3.124280
[LightGBM] [Info] Start training from score -1.037006
[LightGBM] [Info] Start training from score -2.018196
[LightGBM] [Info] Start training from score -3.396873
[LightGBM] [Info] Start training from score -1.957811
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.634976 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col

KeyboardInterrupt: 

In [ ]:
# 保存結果至 CSV
result_df = pd.DataFrame({
    'id': test_data['tweet_id'],
    'predicted_emotion': y_pred
})

result_df.to_csv("prediction_results.csv", index=False)
print("Prediction results saved to prediction_results.csv")

Prediction results saved to prediction_results.csv


In [ ]:
pip install lightgbm xgboost

  Using cached xgboost-2.1.2-py3-none-macosx_10_15_x86_64.macosx_11_0_x86_64.macosx_12_0_x86_64.whl.metadata (2.1 kB)
Using cached xgboost-2.1.2-py3-none-macosx_10_15_x86_64.macosx_11_0_x86_64.macosx_12_0_x86_64.whl (2.1 MB)
Note: you may need to restart the kernel to use updated packages.


### LightGBM

In [ ]:
from lightgbm import LGBMClassifier

# 定义 LightGBM 模型
model = LGBMClassifier(n_estimators=300, random_state=42)

# 训练模型
model.fit(X_train, y_train)

# 测试集预测
y_test_pred_lgbm = model.predict(X_test)

### SVC

In [ ]:
from sklearn.svm import SVC
import pandas as pd

test_data = pd.read_pickle('output_1/test_data.pkl')

# 讀取 .pkl 文件
X_train = pd.read_pickle('output_3/X_train.pkl')
y_train = pd.read_pickle('output_3/y_train.pkl')
X_test = pd.read_pickle('output_3/X_test.pkl')
y_test = pd.read_pickle('output_3/y_test.pkl')

# 初始化並訓練支持向量機模型
svm_clf = SVC(random_state=42)
svm_clf.fit(X_train, y_train)

# 進行預測
y_pred = svm_clf.predict(X_test)

# 保存結果至 CSV
result_df = pd.DataFrame({
    'id': test_data['tweet_id'],
    'predicted_emotion': y_pred
})

result_df.to_csv("prediction_results.csv", index=False)
print("Prediction results saved to prediction_results.csv")

## different attempt

### 1.
[first submission](https://github.com/ChenHsuanHsu/DM2024-Lab2-Homework/blob/main/first_try.ipynb)

feature enginering : **Bag of Words**  
model : **DecisionTree**

>score : 0.28376

### 2.
[second submission](https://github.com/ChenHsuanHsu/DM2024-Lab2-Homework/blob/main/second_try.ipynb)

feature enginering : **Bag of Words**  
model : **DecisionTree**

problem: Too much different data, fail to submit

### 3 & 4 attempt
[third submission](https://github.com/ChenHsuanHsu/DM2024-Lab2-Homework/blob/main/third_try.ipynb)

feature enginering : **Bag of Words**  
model3 : **LightGBM**  
  
> score:0.35555

model4 : **XGBoost**  
> score:0.20148  

model5 : **LogisticRegression**  
> score:0.33557   

### 5 attempt  

model6 : **LightGBM**  
> score:0.37052  



other attempt : voting regression